# Sampling and Splitting

## Sampling

Sampling for basic tabular datasets. (Not designed for time series as of now.)

In [1]:
import polars as pl
import polars_ds as pds
import polars_ds.sample as sa

In [2]:
df = pds.random_data(size=100_000, n_cols = 0).with_columns(
    pds.random(0.0, 12.0).alias("uniform_1"),
    pds.random(0.0, 1.0).alias("uniform_2"),
    pds.random_exp(0.5).alias("exp"),
    pds.random_normal(0.0, 1.0).alias("normal"),
    pds.random_normal(0.0, 1000.0).alias("fat_normal"),
    (pds.random_int(0, 3)).alias("flags"),
    pl.Series(["A"] * 30_000 + ["B"] * 30_000 + ["C"] * 40_000).alias("category"),
)
df.head()

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,0.935568,0.119996,0.019354,-0.047133,144.690433,2,"""A"""
1,9.679801,0.05313,0.31481,-0.448221,-55.713233,2,"""A"""
2,4.627169,0.592026,0.211717,0.256879,-688.904619,1,"""A"""
3,6.670114,0.292098,1.031165,-0.80908,793.655198,0,"""A"""
4,1.357895,0.968076,1.663537,-0.17421,728.835809,2,"""A"""


In [3]:
sa.random_cols(df, 2, keep = ["row_num"])

['row_num', 'exp', 'normal']

In [4]:
# Random Sample
sa.sample(df, 0.6) # by ratio

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,0.935568,0.119996,0.019354,-0.047133,144.690433,2,"""A"""
3,6.670114,0.292098,1.031165,-0.80908,793.655198,0,"""A"""
4,1.357895,0.968076,1.663537,-0.17421,728.835809,2,"""A"""
5,3.30599,0.113208,0.078005,0.975963,678.703558,0,"""A"""
6,10.069914,0.199445,2.185726,0.190404,1028.748911,2,"""A"""
…,…,…,…,…,…,…,…
99992,9.25954,0.815892,2.256677,-0.960783,285.121927,2,"""C"""
99993,8.097766,0.218478,1.992431,0.048365,587.851111,2,"""C"""
99995,0.307021,0.64763,1.934077,1.155237,137.697902,0,"""C"""


In [5]:
sa.sample(df, 30_000) # by count

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
1,9.679801,0.05313,0.31481,-0.448221,-55.713233,2,"""A"""
2,4.627169,0.592026,0.211717,0.256879,-688.904619,1,"""A"""
4,1.357895,0.968076,1.663537,-0.17421,728.835809,2,"""A"""
8,10.006869,0.59061,3.405255,1.089997,293.607536,2,"""A"""
10,11.277722,0.965015,0.20138,1.191682,255.865957,2,"""A"""
…,…,…,…,…,…,…,…
99986,6.633542,0.002842,1.524212,-0.752298,127.119082,2,"""C"""
99989,3.554646,0.319168,0.212673,-0.429877,1373.347225,0,"""C"""
99991,11.707947,0.175283,3.409857,0.198862,2649.120254,0,"""C"""


In [6]:
df.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,33257
1,33270
2,33473


In [7]:
# Downsample on one group
sa1 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5)
)
sa1.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16629
1,33270
2,33473


In [8]:
# Downsample on multiple groups
sa2 = sa.downsample(
    df, 
    [
        (pl.col("flags") == 0, 0.5),
        (pl.col("flags") == 1, 0.3),
        (pl.col("flags") == 2, 0.4),
    ]
)
sa2.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16629
1,9981
2,13390


In [9]:
df.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",40000


In [10]:
# Volume neutral by each category, will take the greatest possible value so that we get neutral volume.
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",30000


In [11]:
# Volume neutral (10_000) by each category
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
    target_volume = 10_000
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",10000
"""C""",10000


In [12]:
# Volume neutral (10_000) by a more complicated condition
vn = sa.volume_neutral(
    df,
    by = pl.col("category") == "A",
    target_volume = 10_000
) # This makes sense because count for B + count for C = 10_000
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",4309
"""C""",5691


In [13]:
# Volume neutral sample with a control level. Volume neutral happens under the category level, meaning
# the volume for each flag in each category is neutral.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category")
) 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9898
"""A""",1,9898
"""A""",2,9898
"""B""",0,9963
"""B""",1,9963
"""B""",2,9963
"""C""",0,13283
"""C""",1,13283
"""C""",2,13283


In [14]:
# We may not meet the target volume for all categories.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category"),
    target_volume= 10_000
) # 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9898
"""A""",1,9898
"""A""",2,9898
"""B""",0,9963
"""B""",1,9963
"""B""",2,9963
"""C""",0,10000
"""C""",1,10000
"""C""",2,10000


## Splitting

To be added...